In [61]:
#general instructions

#2 or more patient encounters are required as the authors remove the last encounter while making predictions
# patient mortality is the outcome label and aquired from mimic. 
#2825 out of 7537 patients died (37.9%)
#Data set was divided into  train (5275/7537), validation (753/7537) and test (1509/7537)
# Once the optimal parameters were selected, the model was retrained by combining train and validation (6028/7537)
#
#
#********************** stars mean the data is not correct but the implemention functions.
#

In [62]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import time
import sys
import pickle
import math
import argparse
from tqdm import tqdm
from __future__ import division
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

In [63]:
# set seed - only really necessary to compare results with each other, but can remove from the submission
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

# define data path
DATA_PATH = "~/Documents/DLH 598/mimic-iii-clinical-database-1.4"


# define data - separated to show which modules load (some take much longer than others)
#The commented out sections take a long time to load, and weren't immidiately useful at this stage
#Can uncommment when relevant

#TODO
# - reorganize into a definition() after  

In [64]:
Patients = pd.read_csv(DATA_PATH + '/PATIENTS.csv')

In [65]:
Diagnosis = pd.read_csv(DATA_PATH + '/DIAGNOSES_ICD.csv')

In [66]:
Procedures = pd.read_csv(DATA_PATH + '/D_ICD_PROCEDURES.csv')

In [67]:
#Prescriptions = pd.read_csv(DATA_PATH + '/PRESCRIPTIONS.csv')

In [68]:
#LabEvents = pd.read_csv(DATA_PATH + '/LABEVENTS.csv')

In [69]:
LabLabels = pd.read_csv(DATA_PATH + '/D_LABITEMS.csv')

In [70]:
Admissions = pd.read_csv(DATA_PATH + '/ADMISSIONS.csv')

In [71]:
ProcedureLabels = pd.read_csv(DATA_PATH + '/D_ITEMS.csv')

In [72]:
IcuStays = pd.read_csv(DATA_PATH + '/ICUSTAYS.csv')

In [73]:
#ChartEvents = pd.read_csv(DATA_PATH + '/CHARTEVENTS.csv')

In [74]:
# patient inclusion - Only ICU patients with 2 or more visits are included. Patient IDs gathered
#May be incorrect as the paper specifies only 7,537 patients in their list.
pd.set_option('display.max_rows', None)
np.set_printoptions(threshold=sys.maxsize)

#********************************************
#Generate an ignore list for later ease of use.
#Manually verified that all the included patients have 2 or more subject ids. Refine by HADM_ID? ICUSTAY_ID?

def Patient_Inclusion(IcuStays):
    IcuList = IcuStays.to_numpy()

    unique, counts = np.unique(IcuList[1:, 1], return_counts=True)
    
    #IcuStayList = EthStatistics[EthStatistics.SUBJECT_ID.isin(Patient_ID)]
    
    print("Max encouters should equal 42?", "max counts: ", max(counts))
    
    AllPatients = np.array(list(zip(np.transpose(unique), np.transpose(counts))))
    
    PatientList = []
    IgnoreList = []
    
    for i in AllPatients:
        if i[1] >= 2:
            PatientList.append(i)
        else:
            IgnoreList.append(i[0])
            
            
    uniquehadm, counts2 = np.unique(IcuList[1:, 1], return_counts=True)
    
    
            
    hadm_ID = IcuStays[['SUBJECT_ID', 'HADM_ID']]
    
    IcuStay_ID = IcuStays[['SUBJECT_ID', 'ICUSTAY_ID']]
    
    unique2= pd.DataFrame(PatientList, columns = ["SUBJECT_ID", "Count"])

    #print(len(PatientList))
    
    HadmIcuList = hadm_ID[hadm_ID.SUBJECT_ID.isin(unique2.SUBJECT_ID)]
    HadmIcuList = HadmIcuList.drop_duplicates("HADM_ID")
    uniquehadm, counts2 = np.unique(IcuList[1:, 1], return_counts=True)
    
    hadm2 = np.array(list(zip(np.transpose(uniquehadm), np.transpose(counts2))))
    hadmlist, nothadm = [], []
    for i in hadm2:
        if i[1] == 1:
            hadmlist.append(i)
        else:
            nothadm.append(i[0])
    
    
    #print(len(hadmlist))
    
    #print(hadmlist)
    
    
    
    #HadmIcu = HadmIcuList[HadmIcuList.SUBJECT_ID.isin(hadm_ID.SUBJECT_ID)]
    
    #print(HadmIcuList)
                

    #print(len(IgnoreList))
    return PatientList, IgnoreList

In [75]:
#To be added to main at the end


#separate patient ids and count information
PatientList, IgnoreList = Patient_Inclusion(IcuStays)
PatientList = pd.DataFrame(PatientList, columns = ['ID', 'Count'])
Patient_ID = PatientList['ID']
Visits = PatientList['Count']

Max encouters should equal 42? max counts:  41


In [76]:
# data statistics - Ethnicity and visit statistics

def Ethnicity_Statistics(Admissions):
    white = 0
    black = 0
    hispanic = 0
    asian = 0
    other = 0
    
    #separate out ethnicity information
    EthStatistics = Admissions[['SUBJECT_ID', 'ETHNICITY']]
    IcuPatientEth = EthStatistics[EthStatistics.SUBJECT_ID.isin(Patient_ID)].drop_duplicates('SUBJECT_ID').reset_index(drop = True)

    #Not yet working.... General idea.
    for i in IcuPatientEth.ETHNICITY:
        if "WHITE" in i:
            white += 1
        if "BLACK" in i:
            black += 1
        if "HISPANIC" in i:
            hispanic += 1
        if "ASIAN"  in i:
            asian += 1
    
        #I tried combining it, but the count was always more wrong...
        if "OTHER" in i:
            other += 1
        if "DECLINED" in i:
            other += 1
        if "NATIVE" in i:
            other += 1
        if "MIDDLE EASTERN" in i:
            other += 1
        if "MULTI" in i:
            other += 1
        if "UNABLE" in i:
            other += 1
        if "UNKNOWN" in i:
            other += 1
        

    print(white / len(IcuPatientEth), "percent white")
    print(black / len(IcuPatientEth), "percent black")
    print(hispanic / len(IcuPatientEth), "percent hispanic")
    print(asian / len(IcuPatientEth), "percent asian")
    print(other / len(IcuPatientEth), "percent other")

    print(((white + black + hispanic + asian + other) / len(IcuPatientEth)), "percent total")
    
    #*****************************************************
    #6 too many values. Need to sort that out.... maybe? it doesnt actually really affect anything...
    print(len(IcuPatientEth))
    print(white + black + asian + hispanic + other)
    return white, black, hispanic, asian, other

In [77]:
#To be added to main at the end
Ethnicity_Statistics(Admissions)

0.7356938892061679 percent white
0.10428326670474015 percent black
0.03438035408338092 percent hispanic
0.024785836664762993 percent asian
0.10154197601370646 percent other
1.0006853226727583 percent total
8755
8761


(6441, 913, 301, 217, 889)

In [78]:
#Age statistics - Date of birth DOB is in patients, and admissions time is in admissions.
# need to correlate both to find patients true ages
#Sorting to ensure that each patient id is matched in case of misordering
def Age_statistics(Patients):

    #separate out date of birth information and sort
    AgeStatistics = Patients[['SUBJECT_ID', 'DOB']]
    IcuPatientDob = AgeStatistics[AgeStatistics.SUBJECT_ID.isin(Patient_ID)].drop_duplicates('SUBJECT_ID').reset_index(drop = True)
    IcuPatientDob = IcuPatientDob.sort_values(by=['SUBJECT_ID']).reset_index(drop = True)
    
    #separate out admission date and sort
    AdmStatistics = Admissions[['SUBJECT_ID', 'ADMITTIME']]
    IcuPatientAdm = AdmStatistics[AdmStatistics.SUBJECT_ID.isin(Patient_ID)].drop_duplicates('SUBJECT_ID').reset_index(drop = True)
    IcuPatientAdm = IcuPatientAdm.sort_values(by=['SUBJECT_ID']).reset_index(drop = True)
    
    #minus dates to find ages
    #********************************************************
    # not done yet. need to convert at least part of the date to an int for each dataframe
    #Year and month should be sufficient, but year month date would increase accuracy. remove time.
    #PatientAges = IcuPatientAdm.ADMITTIME - IcuPatientDob.DOB

    
    
Age_statistics(Patients)

In [79]:
#Sex statistics
def Gender_Statistics(Patients):
    male = 0
    female = 0
    
    SexStatistics = Patients[['SUBJECT_ID', 'GENDER']]
    IcuPatientSex = SexStatistics[SexStatistics.SUBJECT_ID.isin(Patient_ID)].drop_duplicates('SUBJECT_ID').reset_index(drop = True)
    
    for i in IcuPatientSex.GENDER:
        if i == "M":
            male += 1
        if i == "F":
            female += 1
            
    print(male)
    print(female)
    
    print(male / len(IcuPatientSex), " percent male")
    print(female / len(IcuPatientSex), "percent female")
    
    return male, female

In [80]:
Gender_Statistics(Patients)

4943
3812
0.5645916619074814  percent male
0.43540833809251855 percent female


(4943, 3812)

In [81]:
#ICD Code statistics
def Icd_Codes(Diagnosis):
    
    #Isolate just the ICD codes and sequence of ICD codes
    IcdStatistics = Diagnosis[['SUBJECT_ID', 'ICD9_CODE', 'SEQ_NUM']]
    print(IcdStatistics.SEQ_NUM.head(20))
    IcdStatistics.SEQ_NUM = IcdStatistics.SEQ_NUM.astype(int)
    print(IcdStatistics.SEQ_NUM.head(20))
    IcuPatientIcd = IcdStatistics[IcdStatistics.SUBJECT_ID.isin(Patient_ID)].reset_index(drop = True)
    
    #For overall ICD Statistics
    EncounterTotal = IcuPatientIcd.drop_duplicates(['SUBJECT_ID', 'SEQ_NUM']).groupby('SUBJECT_ID')['SEQ_NUM'].count()
    EncounterMax = max(EncounterTotal)
    EncounterMean = np.mean(EncounterTotal)
    EncounterSTD = np.std(EncounterTotal)
    print(EncounterMax)
    print(EncounterMean)
    print(EncounterSTD)
    
    
    #For ICD code groupings by patient id
    IcdGroup = IcuPatientIcd.groupby("SUBJECT_ID")["ICD9_CODE"]

    for key, item in IcdGroup:
        print(IcdGroup.get_group(key), "\n\n")
    
    
Icd_Codes(Diagnosis)

0      1.0
1      2.0
2      3.0
3      4.0
4      5.0
5      6.0
6      7.0
7      8.0
8      9.0
9     10.0
10    11.0
11    12.0
12    13.0
13    14.0
14     1.0
15     2.0
16     3.0
17     4.0
18     5.0
19     6.0
Name: SEQ_NUM, dtype: float64


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
#Medication statistics
def Medications():
    #Need advice about medications for data science uses. In nursing, each medication is necessary 
    #however the authors mension removing duplicates per encounter. Should we only be keeping one medication
    #type per encounter? This doesnt seem like a good idea, but I am not sure if that is what the author is 
    #implying. Or, should we keep one medication type per day?
    pass

In [ ]:
#Lab component statistics
#The paper mentioned significant removal of duplicate patient information at this step (20% of data removed)
#a total of 5,609,021 lab tests were removed as they were not used.

def Laboratory():
    #Only include the abnormal flags, and ignore the actual values.
    LabStatistics = LabEvents[['SUBJECT_ID', 'ITEMID', 'FLAG']]
    LabStatistics.SEQ_NUM = LabStatistics.SEQ_NUM
    IcuPatientLab = LabStatistics[LabStatistics.SUBJECT_ID.isin(Patient_ID)].reset_index(drop = True)

    #

In [ ]:
#Outcome label = patient mortatility. Flag found in Admissions?

#Right idea, may not be correct
#**************************************************

OutcomeFlag = Admissions[['SUBJECT_ID', 'HOSPITAL_EXPIRE_FLAG']]
Outcome = OutcomeFlag[OutcomeFlag.SUBJECT_ID.isin(Patient_ID)].drop_duplicates('SUBJECT_ID').reset_index(drop = True)
Deceased = []


for i in Outcome.to_numpy():
    if i[1] == 1:
        Deceased.append(i[0])
        
Deceased = np.array(Deceased)
pd.DataFrame(Deceased, columns = ['Deceased'])  


print(Deceased)

In [ ]:
#Data loader


In [ ]:
#Retain
#just the skeleton from the assignment. Complete but not yet tested or functional. Need to mold data to this first.
#**************************************************************

# Not implemented but may be better than the course work assignment model!!! That being said, there is no attention
# so it may just be an RNN instead of true Retain
#***************************************************************************
# Alternate model from github forr CLOUT: https://github.com/subendhu19/CLOUT/blob/master/rnn_icd.py

In [ ]:
#Calculate alpha attention

class AlphaAttention(torch.nn.Module):
    def __init__(self, hidden_dimension):
        super().__init__()
        self.Alpha_Attention = nn.Linear(hidden_dimension, 1)
        
    def forward(self, gi):
        x = self.Alpha_Attention(gi)
        softmax = torch.nn.Softmax(dim = 1)
        x = softmax(x)
        return x

In [ ]:
#Calculate beta attention

class BetaAttention(torch.nn.Module):
    def __init__(self, hidden_dimension):
        super().__init__()
        self.Beta_Attention = nn.Linear(hidden_dimension, hidden_dimension)
        
    def forward(self, hi):
        x = self.Beta_Attention(hi)
        x = torch.tanh(x)
        return x

In [ ]:
#Sum the alpha and beta attention together.

def AttentionSum(alpha, beta, reverse_v, reverse_masks):
    rev_masks = torch.unsqueeze(torch.max(reverse_masks, dim = 2)[0], dim = -1)
    c = alpha * beta * (rev_v * reverse_masks)
    c = torch.sum(c, dim = 1)
    return c

In [ ]:
#sum embeddings with the mask

def SumEmbeddingsWithMask(x, masks):
    x = x * masks.unsqueeze(-1)
    x = torch.sum(x, dim = -2)
    return x

In [ ]:
# Build the retain model
#Diagnosis codes are a dictionary of diagnosis codes with a numerical value of its order following
#ex. diag_101 : 0, diag_103 : 1, diag_187 : 2 ....

class Retain(nn.Module):
    def __init__(self, diagnosis_codes, embedding_dimension = 128):
        super().__init__()
        self.embedding = nn.Embedding(diagnosis_codes, embedding_dimension)
        self.rnn_alpha = nn.GRU(embedding_dimension, embedding_dimension, batch_first = True)
        self.rnn_beta = nn.GRU(embedding_dimension, embedding_dimension, batch_first = True)
        self.alpha_attention = AlphaAttention(embedding_dimension)
        self.beta_attention = BetaAttention(embedding_dimension)
        self.fully_connected = nn.Linear(embedding_dimension, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, masks, reverse_x, reverse_masks):
        reverse_x = self.embedding(reverse_x)
        reverse_x = SumEmbeddingsWithMask(reverse_x, reverse_masks)
        gi, _ = self.rnn_alpha(reverse_x)
        hi, _ = self.rnn_beta(reverse_x)
        alpha = self.alpha_attention(gi)
        beta = self.beta_attention(hi)
        context = AttentionSum(alpha, beta, reverse_x, reverse_masks)
        logits = self.fully_connected(context)
        probability = self.sigmoid(logits).squeeze()
        
        return probability

retain = Retain(diagnosis_codes = len(#******************** not implemented yet. part of data loader))
retain

In [ ]:
#Retain evaluation

def eval(model, value_loader):
    model.eval()
    y_pred = torch.LongTensor()
    y_score = torch.Tensor()
    y_true = torch.LongTensor()
    model.eval()
    for x, masks, reverse_x, reverse_masks, y in value_loader:
        y_logit = model(x, masks, reverse_x, reverse_masks)
        y_hat = y_logit > 0.5
        
        y_score = torch.cat((y_score, y_logit.detach().to('cpu')), dim = 0)
        y_pred = torch.cat((y_pred, y_hat.detach().to('cpu')), dim = 0)
        y_true torch.cat((y_true, y.detach().to('cpu')), dim = 0)
        
    p, r, f, _ = precision_recall_fscore_support(y_true, y_pred, average = 'binary')
    roc_auc = roc_auc_score(y_true, y_score)
    return p, r, f, roc_auc

In [ ]:
#Retain training

def train(model, train_loader, value_loader, epochs):
    for epoch in range(epochs):
        model.train()
        training_loss = 0
        for x, masks, reverse_x, reverse_masks, y in traing_loader:
            optimizer.zero_grad()
            y_hat = model(x, masks, reverse_x, reverse_masks)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            training_loss += loss.item()
        training_loss = training_loss / len(train_loader)
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
        p, r, f, roc_auc = eval(model, value_loader)
        print('Epoch: {} \t Validation p: {:.2f}, r:{:.2f}, f: {:.2f}, roc_auc: {:.2f}'.format(epoch+1, p, r, f, roc_auc))
        
    return round(roc_auc, 2)

In [ ]:
#add to main function at the end

#this line is redundant? may remove as needed or remove the other line above.
retain = Retain(diagnosis_codes = len(#***************************not implemented yet))
    
criterion = nn.BCELoss()

optimizer = torch.optim.Adam(retain.parameters(), lr = 1e-3)
    
epochs = 10
train(retain, train_loader, value_loader, epochs)

In [ ]:
#Clout model (The most important model to finish)

#Source: https://github.com/subendhu19/CLOUT

In [ ]:
# Clout outline photos to work with



In [ ]:
#Clout model

class RNN(nn.Module):
    def __init__(self, epochs=5, batchsize=50, vocabsize=5, embsize=100):
        super(RNN, self).__init__()
        self.epochs = 5
        self.batchsize = batchsize
        self.vocabsize = vocabsize
        self.embsize = embsize

        self.emb_icd = nn.Linear(vocabsize_icd, embsize_icd)
        self.emb_meds = nn.Linear(vocabsize_meds, embsize_meds)
        self.emb_labs = nn.Linear(vocabsize_labs, embsize_labs)

        self.rnn = nn.LSTM(input_size=embsize, hidden_size=embsize, num_layers=1)
        self.out = nn.Linear(embsize, 1)
        self.sig = nn.Sigmoid()

    def forward(self, input_icd, input_med, input_lab, input_latent, hidden=None, force=True, steps=0):
        if force or steps == 0: steps = len(input_icd)
        outputs = Variable(torch.zeros(steps, 1, 1))

        input_icd = self.emb_icd(input_icd)
        input_med = self.emb_meds(input_med)
        input_lab = self.emb_labs(input_lab)

        inputs = F.relu(torch.cat((input_icd, input_med, input_lab, input_latent),1))

        inputs = inputs.view(inputs.size()[0],1,inputs.size()[1])
        outputs, hidden = self.rnn(inputs, hidden)
        outputs = self.out(outputs)
        return outputs.squeeze(), hidden

    def predict(self, input_icd, input_med, input_lab, input_latent):
        out, hid = self.forward(input_icd, input_med, input_lab, input_latent, None)
        return self.sig(out[-1]).data[0]

parser = argparse.ArgumentParser(description='Coq tactic prediction')
parser.add_argument('--emb_weights', type=str, default='../full data/CAE/CAE_embedding_weights.npy',
                    help='location of the embedding weight file (default: ../full data/CAE/CAE_embedding_weights.npy)')
args = parser.parse_args()

n_epochs = 1
vocabsize_icd = 942
vocabsize_meds = 3202
vocabsize_labs = 284 #all 681
vocabsize = vocabsize_icd+vocabsize_meds+vocabsize_labs

embsize_icd = 50
embsize_meds = 75
embsize_labs = 50
embsize_latent = 175
embsize = embsize_icd + embsize_labs + embsize_meds + embsize_latent

input_seqs_icd = np.array(pickle.load(open('../full data/MIMICIIIPROCESSED.3digitICD9.seqs')))
input_seqs_meds = np.array(pickle.load(open('../full data/MIMICIIIPROCESSED.meds.seqs')))
input_seqs_labs = np.array(pickle.load(open('../full data/MIMICIIIPROCESSED.abnlabs.seqs')))
# input_seqs_latent = np.array(pickle.load(open('../full data/CAE/Embeddings.seqs')))
latent_weights = pickle.load(open(args.emb_weights))

input_seqs_fullicd = np.array(pickle.load(open('../full data/MIMICIIIPROCESSED.seqs')))

icditems = pickle.load(open('../full data/type dictionaries/MIMICIIIPROCESSED.types', 'rb'))
meditems = pickle.load(open('../full data/type dictionaries/MIMICIIIPROCESSED.meds.types', 'rb'))
labitems = pickle.load(open('../full data/type dictionaries/MIMICIIIPROCESSED.abnlabs.types', 'rb'))

model_name = 'AE'
if args.emb_weights == '../full data/CAE/CAE_embedding_weights.npy':
    model_name = 'CAE'

interpretation_file = open("RNN_CLatent_Interpretations_" + model_name + ".txt", 'w')

# overall_risk_factor_file = open("risk_factors_averaged.txt", "w")

labnames = {}
lab_dict_file = open('D_LABITEMS.csv', 'r')
lab_dict_file.readline()
for line in lab_dict_file:
    tokens = line.strip().split(',')
    labnames[tokens[1].replace('"','')] = tokens[2]
lab_dict_file.close()

icdnames = {}
icd_dict_file = open('D_ICD_DIAGNOSES.csv', 'r')
icd_dict_file.readline()
for line in icd_dict_file:
    tokens = line.strip().split(',')
    icdnames[tokens[1].replace('"','')] = tokens[2]
icd_dict_file.close()

icd_scores = {}
med_scores = {}
lab_scores = {}

icd_totals = {}
med_totals = {}
lab_totals = {}

def get_ICD(icd):
    ret_str = ""
    icd_str = icditems.keys()[icditems.values().index(icd)]
    actual_key = icd_str.replace(".", "")[2:]
    if actual_key in icdnames:
        ret_str = icdnames[actual_key]
    else:
        ret_str = icd_str
    return ret_str

def get_med(med):
    ret_str = meditems.keys()[meditems.values().index(med)]
    return ret_str

def get_lab(lab):
    ret_str = labnames[labitems.keys()[labitems.values().index(lab)]]
    return ret_str

print 'Data loaded..'

labels = np.array(pickle.load(open('../full data/MIMICIIIPROCESSED.morts')))

trainratio = 0.7
validratio = 0.1
testratio = 0.2

trainlindex = int(len(input_seqs_icd)*trainratio)
validlindex = int(len(input_seqs_icd)*(trainratio + validratio))

print 'Data prepared..'

def convert_to_one_hot(code_seqs, len):
    new_code_seqs = []
    for code_seq in code_seqs:
        one_hot_vec = np.zeros(len)
        for code in code_seq:
            one_hot_vec[code] = 1
        new_code_seqs.append(one_hot_vec)
    return np.array(new_code_seqs)

def get_avg(seqs, type):
    count = 0
    for seq in seqs:
        count += len(seq)
    val = round(count*1.0/len(seqs))
    if type == 'i':
        return min(4, int(val/5))
    else:
        return min(4, int(val/50))

def get_factors(icd_seq, med_seq, lab_seq, model, actual_score, full_icd):
    potential_test_data = []

    for seq in range(len(icd_seq)):
        for i in range(len(icd_seq[seq])):
            potential_test_data.append(("icd", full_icd[seq][i], seq, icd_seq[:seq]+[icd_seq[seq][:i] + icd_seq[seq][i+1:]]+icd_seq[seq+1:], med_seq, lab_seq))
    for seq in range(len(med_seq)):
        for i in range(len(med_seq[seq])):
            potential_test_data.append(("med", med_seq[seq][i], seq, icd_seq, med_seq[:seq]+[med_seq[seq][:i]+med_seq[seq][i+1:]]+med_seq[seq+1:], lab_seq))
    for seq in range(len(lab_seq)):
        for i in range(len(lab_seq[seq])):
            potential_test_data.append(("lab", lab_seq[seq][i], seq, icd_seq, med_seq, lab_seq[:seq]+[lab_seq[seq][:i] + lab_seq[seq][i+1:]]+lab_seq[seq+1:]))

    risk_scores = []

    for pt in potential_test_data:
        test_input_icd = Variable(torch.from_numpy(convert_to_one_hot(pt[3], vocabsize_icd)).float())
        test_input_med = Variable(torch.from_numpy(convert_to_one_hot(pt[4], vocabsize_meds)).float())
        test_input_lab = Variable(torch.from_numpy(convert_to_one_hot(pt[5], vocabsize_labs)).float())

        latent_inputs_oh = np.concatenate((convert_to_one_hot(pt[3], vocabsize_icd), convert_to_one_hot(pt[4], vocabsize_meds), convert_to_one_hot(pt[5], vocabsize_labs)), 1)
        latent_inputs = np.dot(latent_inputs_oh, latent_weights)
        latent_inputs = Variable(torch.from_numpy(latent_inputs).float())

        factor_score = actual_score - model.predict(test_input_icd, test_input_med, test_input_lab, latent_inputs)
        factor = ""
        if pt[0] == 'icd':
            icd_tag = get_ICD(pt[1])
            factor = "ICD-"+icd_tag
            if icd_tag in icd_scores:
                icd_scores[icd_tag] += factor_score
                icd_totals[icd_tag] += 1
            else:
                icd_scores[icd_tag] = factor_score
                icd_totals[icd_tag] = 1
        elif pt[0] == 'med':
            med_tag = get_med(pt[1])
            factor = "Med-"+med_tag
            if med_tag in med_scores:
                med_scores[med_tag] += factor_score
                med_totals[med_tag] += 1
            else:
                med_scores[med_tag] = factor_score
                med_totals[med_tag] = 1
        else:
            lab_tag = get_lab(pt[1])
            factor = "Lab-"+lab_tag
            if lab_tag in lab_scores:
                lab_scores[lab_tag] += factor_score
                lab_totals[lab_tag] += 1
            else:
                lab_scores[lab_tag] = factor_score
                lab_totals[lab_tag] = 1
        risk_scores.append(("Encounter-"+str(pt[2])+": "+factor, factor_score))

    risk_scores.sort(key=lambda tup: tup[1], reverse=True)

    return risk_scores[:10]

print 'Starting training..'

batchsize = 50

# ICD_wise_tot_tr = np.zeros(5)
# meds_wise_tot_tr = np.zeros(5)
# labs_wise_tot_tr = np.zeros(5)

# for i in range(len(train_input_seqs_icd)):
# 	ICD_wise_tot_tr[get_avg(train_input_seqs_icd[i], 'i')] += 1
# 	meds_wise_tot_tr[get_avg(train_input_seqs_meds[i], 'm')] += 1
# 	labs_wise_tot_tr[get_avg(train_input_seqs_labs[i], 'l')] += 1

# print 'ICD-wise train total', ICD_wise_tot_tr
# print 'Meds-wise train total', meds_wise_tot_tr
# print 'Labs-wise train total', labs_wise_tot_tr

best_aucrocs = []
for run in range(10):
    print 'Run', run

    perm = np.random.permutation(input_seqs_icd.shape[0])
    rinput_seqs_icd = input_seqs_icd#[perm]
    rinput_seqs_meds = input_seqs_meds#[perm]
    rinput_seqs_labs = input_seqs_labs#[perm]
    # rinput_seqs_latent = input_seqs_latent[perm]
    rinput_seqs_fullicd = input_seqs_fullicd#[perm]
    rlabels = labels#[perm]

    train_input_seqs_icd = rinput_seqs_icd[:trainlindex]
    train_input_seqs_meds = rinput_seqs_meds[:trainlindex]
    train_input_seqs_labs = rinput_seqs_labs[:trainlindex]
    # train_input_seqs_latent = rinput_seqs_latent[:trainlindex]
    train_labels = rlabels[:trainlindex]
    train_labels = train_labels.reshape(train_labels.shape[0],1)

    valid_input_seqs_icd = rinput_seqs_icd[trainlindex:validlindex]
    valid_input_seqs_meds = rinput_seqs_meds[trainlindex:validlindex]
    valid_input_seqs_labs = rinput_seqs_labs[trainlindex:validlindex]
    # valid_input_seqs_latent = rinput_seqs_latent[trainlindex:validlindex]
    valid_labels = rlabels[trainlindex:validlindex]

    test_input_seqs_icd = rinput_seqs_icd[validlindex:]
    test_input_seqs_meds = rinput_seqs_meds[validlindex:]
    test_input_seqs_labs = rinput_seqs_labs[validlindex:]
    # test_input_seqs_latent = rinput_seqs_latent[validlindex:]
    test_input_seqs_fullicd = rinput_seqs_fullicd[validlindex:]
    test_labels = rlabels[validlindex:]

    n_iters = train_input_seqs_icd.shape[0]

    model = RNN(n_epochs, 1, vocabsize, embsize)
    criterion = nn.BCEWithLogitsLoss(size_average=False)
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    aucrocs = []

    for epoch in range(n_epochs):
        epoch_loss = 0
        print 'Epoch', (epoch+1)

        for i in (range(0, n_iters, batchsize)):
            batch_icd = train_input_seqs_icd[i:i+batchsize]
            batch_meds = train_input_seqs_meds[i:i+batchsize]
            batch_labs = train_input_seqs_labs[i:i+batchsize]
            # batch_latent = train_input_seqs_latent[i:i+batchsize]

            batch_train_labels = train_labels[i:i+batchsize]

            optimizer.zero_grad()
            losses = []

            for i in range(len(batch_icd)):
                icd_onehot = convert_to_one_hot(batch_icd[i], vocabsize_icd)
                med_onehot = convert_to_one_hot(batch_meds[i], vocabsize_meds)
                lab_onehot = convert_to_one_hot(batch_labs[i], vocabsize_labs)

                icd_inputs = Variable(torch.from_numpy(icd_onehot).float())
                med_inputs = Variable(torch.from_numpy(med_onehot).float())
                lab_inputs = Variable(torch.from_numpy(lab_onehot).float())

                latent_inputs_oh = np.concatenate((icd_onehot, med_onehot, lab_onehot), 1)
                latent_inputs = np.dot(latent_inputs_oh, latent_weights)
                latent_inputs = Variable(torch.from_numpy(latent_inputs).float())

                # latent_inputs = Variable(torch.from_numpy(np.array(batch_latent[iter])).float())

                targets = Variable(torch.from_numpy(batch_train_labels[i]).float())

                # Use teacher forcing 50% of the time
                force = random.random() < 0.5
                outputs, hidden = model(icd_inputs, med_inputs, lab_inputs, latent_inputs, None, force)

                #print outputs[-1], targets
                losses.append(criterion(outputs[-1], targets))

            loss = sum(losses)/len(batch_icd)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.data[0]

        #print(epoch, epoch_loss)

        ## Validation phase
        vpredictions = np.zeros(len(valid_input_seqs_icd))
        for i in range(len(valid_input_seqs_icd)):
            test_input_icd = Variable(torch.from_numpy(convert_to_one_hot(valid_input_seqs_icd[i], vocabsize_icd)).float())
            test_input_med = Variable(torch.from_numpy(convert_to_one_hot(valid_input_seqs_meds[i], vocabsize_meds)).float())
            test_input_lab = Variable(torch.from_numpy(convert_to_one_hot(valid_input_seqs_labs[i], vocabsize_labs)).float())

            test_input_latent_oh = np.concatenate((convert_to_one_hot(valid_input_seqs_icd[i], vocabsize_icd), convert_to_one_hot(valid_input_seqs_meds[i], vocabsize_meds), convert_to_one_hot(valid_input_seqs_labs[i], vocabsize_labs)), 1)
            test_input_latent = np.dot(test_input_latent_oh, latent_weights)
            test_input_latent = Variable(torch.from_numpy(test_input_latent).float())

            # test_input_latent = Variable(torch.from_numpy(np.array(valid_input_seqs_latent[i])).float())
            vpredictions[i] = model.predict(test_input_icd, test_input_med, test_input_lab, test_input_latent)

        print "Validation AUC_ROC: ", roc_auc_score(valid_labels, vpredictions)

        ## Testing phase
        predictions = np.zeros(len(test_input_seqs_icd))

        # ICD_wise_corr = np.zeros(5)
        # meds_wise_corr = np.zeros(5)
        # labs_wise_corr = np.zeros(5)
        # ICD_wise_tot = np.zeros(5)
        # meds_wise_tot = np.zeros(5)
        # labs_wise_tot = np.zeros(5)

        for i in range(len(test_input_seqs_icd)):
            test_input_icd = Variable(torch.from_numpy(convert_to_one_hot(test_input_seqs_icd[i], vocabsize_icd)).float())
            test_input_med = Variable(torch.from_numpy(convert_to_one_hot(test_input_seqs_meds[i], vocabsize_meds)).float())
            test_input_lab = Variable(torch.from_numpy(convert_to_one_hot(test_input_seqs_labs[i], vocabsize_labs)).float())

            test_input_latent_oh = np.concatenate((convert_to_one_hot(test_input_seqs_icd[i], vocabsize_icd), convert_to_one_hot(test_input_seqs_meds[i], vocabsize_meds), convert_to_one_hot(test_input_seqs_labs[i], vocabsize_labs)), 1)
            test_input_latent = np.dot(test_input_latent_oh, latent_weights)
            test_input_latent = Variable(torch.from_numpy(test_input_latent).float())

            # test_input_latent = Variable(torch.from_numpy(np.array(test_input_seqs_latent[i])).float())
            predictions[i] = model.predict(test_input_icd, test_input_med, test_input_lab, test_input_latent)
            
            # ICD_wise_corr[get_avg(test_input_seqs_icd[i], 'i')] += int((predictions[i]>0.5)*1 == test_labels[i])
            # ICD_wise_tot[get_avg(test_input_seqs_icd[i], 'i')] += 1

            # meds_wise_corr[get_avg(test_input_seqs_meds[i], 'm')] += int((predictions[i]>0.5)*1 == test_labels[i])
            # meds_wise_tot[get_avg(test_input_seqs_meds[i], 'm')] += 1

            # labs_wise_corr[get_avg(test_input_seqs_labs[i], 'l')] += int((predictions[i]>0.5)*1 == test_labels[i])
            # labs_wise_tot[get_avg(test_input_seqs_labs[i], 'l')] += 1

        print "Test AUC_ROC: ", roc_auc_score(test_labels, predictions)

        aucrocs.append(roc_auc_score(test_labels, predictions))
        #fpr, tpr, _ = roc_curve(test_labels, predictions)
        #pickle.dump({"FPR":fpr, "TPR":tpr}, open('roc_clout_cornn.p', 'wb'))
        #actual_predictions = (predictions>0.5)*1
        #print classification_report(test_labels, actual_predictions)

    best_aucrocs.append(max(aucrocs))

print "Average AUCROC:", np.mean(best_aucrocs), "+/-", np.std(best_aucrocs) 

# print "Final testing and interpretations"
# predictions = np.zeros(len(test_input_seqs_icd))
# for i in (range(len(test_input_seqs_icd))):
# 	test_input_icd = Variable(torch.from_numpy(convert_to_one_hot(test_input_seqs_icd[i], vocabsize_icd)).float())
# 	test_input_med = Variable(torch.from_numpy(convert_to_one_hot(test_input_seqs_meds[i], vocabsize_meds)).float())
# 	test_input_lab = Variable(torch.from_numpy(convert_to_one_hot(test_input_seqs_labs[i], vocabsize_labs)).float())
# 	test_input_latent_oh = np.concatenate((convert_to_one_hot(test_input_seqs_icd[i], vocabsize_icd), convert_to_one_hot(test_input_seqs_meds[i], vocabsize_meds), convert_to_one_hot(test_input_seqs_labs[i], vocabsize_labs)), 1)
# 	test_input_latent = np.dot(test_input_latent_oh, latent_weights)
# 	test_input_latent = Variable(torch.from_numpy(test_input_latent).float())

# 	test_score = model.predict(test_input_icd, test_input_med, test_input_lab, test_input_latent)
# 	predictions[i] = test_score
# 	top_risk_factors = get_factors(test_input_seqs_icd[i], test_input_seqs_meds[i], test_input_seqs_labs[i], model, test_score, test_input_seqs_fullicd[i]) 
# 	if (test_score>0.5):
# 		interpretation_file.write("ID: " + str(i) + " True label: "+str(test_labels[i])+"\n")
# 		for rf in top_risk_factors:
# 			interpretation_file.write(str(rf)+"\n")
# 		interpretation_file.write("\n")

interpretation_file.close()

In [ ]:
#Clout training


In [ ]:
#Clout evaluation


In [ ]:
#Summary conclusions


In [ ]:
#Summary Graphs etc...
